In [1]:
import os
from os.path import join as oj
import sys, time
sys.path.insert(1, oj(sys.path[0], '..'))  # insert parent path
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm, tqdm_notebook
from copy import deepcopy
import pickle as pkl
import pandas as pd
sys.path.append('../models')
sys.path.append('../fit')

from model import LSTMSentiment
from torchtext import data
from torchtext import datasets
import torch
import cd
import pandas as pd

%matplotlib inline
%load_ext autoreload
%autoreload 2

# load dset + model

In [6]:
# data params
vector_cache =  '../data/.vector_cache/input_vectors.pt'
word_vectors ='glove.6B.300d'
batch_size = 50
device = 'cuda' if torch.cuda.is_available() else 'cpu'


# load dset
inputs = data.Field(lower= True)
answers = data.Field(sequential=False, unk_token=None)
train, dev, test = datasets.SST.splits(inputs, answers, fine_grained=False, train_subtrees=False,
                                       filter_pred=lambda ex: ex.label != 'neutral')
inputs.build_vocab(train, dev, test)

if os.path.isfile(vector_cache):
    inputs.vocab.vectors = torch.load(vector_cache)
else:
    inputs.vocab.load_vectors(word_vectors)
    os.makedirs(os.path.dirname(vector_cache), exist_ok=True)
    torch.save(inputs.vocab.vectors, vector_cache)
answers.build_vocab(train)

train_iter, dev_iter, test_iter = data.BucketIterator.splits(
    (train, dev, test), batch_size=batch_size, device=torch.device(0),
                sort_key=lambda x: len(x.text), # the BucketIterator needs to be told what function it should use to group the data.
                         sort_within_batch=True,
                shuffle =True,
    sort = False,
                         repeat=False)

# load model
model_path = "../models/init_models"
model_list = os.listdir(model_path)
model1 = torch.load(os.path.join(model_path, model_list[0]), map_location=torch.device(0)).eval()

#model2 = torch.load(os.path.join(model_path, model_list[1]), map_location=torch.device(0)).eval()

/home/lauri/.local/lib/python3.6/site-packages/torch/serialization.py:454: SourceChangeWarning: source code of class 'model.LSTMSentiment' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/lauri/.local/lib/python3.6/site-packages/torch/serialization.py:454: SourceChangeWarning: source code of class 'torch.nn.modules.sparse.Embedding' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/lauri/.local/lib/python3.6/site-packages/torch/serialization.py:454: SourceChangeWarning: source code of class 'torch.nn.modules.rnn.LSTM' has changed. you can retrieve the original source code by accessing the object's source attribute or 

# evaluate cd word-level

In [7]:
tot_length =0
for i in range(len(train)):
    tot_length +=len(train[i].text)

In [9]:
# choose hyperparams
it = train_iter
m = model1

# what to store
words = {}

it.init_epoch()
# check out how two models differ
import torch.optim as O
import torch.nn as nn
criterion = nn.CrossEntropyLoss()
n_dev_correct, dev_loss = 0, 0

# remember batches are num_words x batch_size
for batch in tqdm_notebook(it):
    answer1 = m(batch)
    num_words = batch.text.shape[0]
    batch_size = batch.text.shape[1]
    
    for word_num in range(num_words-1):
        word_per_batch = batch.text[word_num] # gets word at same place for all batches
  
        # get cd scores for each word
        rel, _ = cd.cd_batch_text(batch, m, start=word_num, stop=word_num + 1)
        rel = rel.softmax(dim=0) # 2 x batch_size
        rel = rel[0] # only get positive class
        
        # actually get the words
        for batch_num in range(word_per_batch.shape[0]):
            word = inputs.vocab.itos[word_per_batch[batch_num]] 
          
            score = rel[batch_num].item()
            
            # add to store
            if not word in words:
                words[word] = (1, score) # count, sum
            else:
                (count, running_sum) = words[word]
                words[word] = (count + 1, running_sum + score)


# look at fairness results

In [2]:
import operator

In [3]:
sorted_x = sorted(words.items(), key=operator.itemgetter(1))

NameError: name 'words' is not defined

In [4]:
test = [x for x in sorted_x if inputs.vocab.stoi[x[0]] !=0]

NameError: name 'sorted_x' is not defined

In [5]:
results = pd.DataFrame()
results['word'] = words.keys()
results['count'] = [words[word][0] for word in results['word']]
results['sent'] = [words[word][1] / words[word][0] for word in results['word']]
results = results.sort_values(by=['sent'], ascending=False)
results.to_pickle('../results/word_fairness_test.pkl')

NameError: name 'words' is not defined

In [7]:
df = pd.read_pickle('../results/word_fairness_test.pkl')
# r = r[r['sent'] <= .40]

In [8]:
df.sort_values(by = ['count'], ascending=False)

,word,count,sent
2,the,5954,0.528480
58,",",5883,0.526845
0,a,4361,0.539533
98,and,3831,0.510247
47,of,3631,0.515243
99,to,2438,0.539548
37,is,2095,0.512990
32,'s,2025,0.513493
1,it,1947,0.520693
1184,.,1808,0.528304


In [16]:
comparisons = [('actor', 'actress'), ('he', 'she'),('him', 'her'), ]
for (x, y) in comparisons:
    print(r[r.word == x], '\n', r[r.word == y], '\n')

       word  count      sent
1128  actor     26  0.334335 
          word  count      sent
4705  actress     16  0.588069 

    word  count      sent
482   he    163  0.623058 
      word  count      sent
1483  she     55  0.600758 

    word  count      sent
353  him     45  0.486916 
      word  count      sent
1161  her    109  0.529266 



In [17]:
len(train)

6920

In [18]:
350/6920

0.05057803468208093